In [2]:
!pip install --upgrade opencv-python transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 70.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4


In [1]:
!pip install --upgrade  git+https://github.com/huggingface/diffusers.git
!pip install --upgrade  git+https://github.com/huggingface/accelerate.git

  Cloning https://github.com/huggingface/diffusers.git to /tmp/pip-req-build-j_5u4zky
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers.git /tmp/pip-req-build-j_5u4zky
  Resolved https://github.com/huggingface/diffusers.git to commit 0c1e63bd11a5746db8933111a962854fa9b36582
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.31.0.dev0-py3-none-any.whl size=2658778 sha256=65a06d45e6b6a4c073df3c2fa569d638d75048b3c66dbdc845e2ace15ca9d5de
  Stored in directory: /tmp/pip-ephem-wheel-cache-nq_i8xk6/wheels/4d/b7/a8/6f9549ceec5daad78675b857ac57d697c387062506520a7b50
Successfully built diffusers
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-hfp9eg57
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /tmp/pip-req-build-hfp9eg57
  Resolve

In [1]:
import gc
import torch
from diffusers import CogVideoXPipeline, CogVideoXDDIMScheduler
from diffusers.utils import export_to_video

def reset_memory():
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.reset_accumulated_memory_stats()
    torch.cuda.reset_peak_memory_stats()

def print_memory():
    memory = round(torch.cuda.memory_allocated() / 1024**3, 2)
    max_memory = round(torch.cuda.max_memory_allocated() / 1024**3, 2)
    max_reserved = round(torch.cuda.max_memory_reserved() / 1024**3, 2)
    print(f"{memory=} GB")
    print(f"{max_memory=} GB")
    print(f"{max_reserved=} GB")

prompt = (
    "A cat walking through the forest sees a bear and gets scared and starts running through a muddy path that is along the middle of the forest , the path is in the middle and at both sides are trees, the bear is chasing the cat, we see the point of view of the cat as the camera is facing the cat and in the background we can see an angry bear chasing down the cat along the muddy path in the middle of the forest.	"
)

pipe = CogVideoXPipeline.from_pretrained("THUDM/CogVideoX-2b", torch_dtype=torch.float16)
pipe.scheduler = CogVideoXDDIMScheduler.from_config(pipe.scheduler.config, timestep_spacing="trailing")

pipe.enable_sequential_cpu_offload()  # Changed from enable_model_cpu_offload
pipe.vae.enable_tiling()

reset_memory()

# Reduce video parameters
with torch.cuda.amp.autocast():  # Enable automatic mixed precision
    video = pipe(
        prompt=prompt,
        num_frames=48,  # Reduced from 48
        guidance_scale=6,
        num_inference_steps=30,  # Reduced from 50
        generator=torch.Generator().manual_seed(42)
    ).frames[0]

print_memory()
export_to_video(video, "output_tiling.mp4", fps=8)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The config attributes {'mid_block_add_attention': True, 'sample_size': 256} were passed to AutoencoderKLCogVideoX, but are not expected and will be ignored. Please verify your config.json configuration file.


  0%|          | 0/30 [00:00<?, ?it/s]

memory=0.07 GB
max_memory=8.72 GB
max_reserved=10.48 GB


/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(


'output_tiling.mp4'